## Imports

In [1]:
from llama_index import download_loader, SimpleDirectoryReader
from ragas.testset import TestsetGenerator
from llama_index import download_loader

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

/opt/anaconda3/envs/ragas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt

## Assessing current state of test date generation

In [3]:

ArxivReader = download_loader("ArxivReader")

loader = ArxivReader()

In [4]:
documents = loader.load_data("abs:LLM",max_results=50)

In [3]:
test_dist =  {
    "simple": 0.3,
    "reasoning": 0.25,
    "multi_context": 0.25,
    "conditional": 0.2,
}

In [4]:
testsetgenerator = TestsetGenerator.from_default(testset_distribution=test_dist,chunk_size=1024)


In [8]:
test_size = 25
testset = testsetgenerator.generate(documents, test_size=test_size)
test_df = testset.to_pandas()
test_df.head()

  0%|                                                            | 0/25 [00:00<?, ?it/s]

{'score': True, 'is_table_present': False}
seed question What are the limitations of LOGIC -LM?
{'score': True, 'is_table_present': True}
seed question What are the four broader application areas of LLMs investigated in the included studies?
{'score': True, 'is_table_present': False}
seed question How do we propose to attack the detectors?
{'score': True, 'is_table_present': True}
seed question What is the metric type for the task "Knowledge Question Answering" in the Legal Knowledge Memorization category?



KeyboardInterrupt



In [ ]:
test_df.to_csv("arxiv_questions_v3.csv")

**Observations**
- Conditional question evol is working
- reasoning/multi context are not working as expected
- Almost all questions are closed endeded 
- Almost all questions start with "What"

## Experiments

In [5]:
from llama_index.readers.file.markdown_reader import MarkdownReader
from llama_index.schema import Document
from typing import List, Dict, Optional
from pathlib import Path
import re
import os

In [6]:
class RagasMdReader(MarkdownReader):
    
    @staticmethod
    def get_file_metadata(path):
        
        return {"filename":os.path.basename(path),
                "dirname":os.path.dirname(path)}
        
    
    def get_local_metadata(self, text):
        
        hyperlinks = {}
        pattern = r"\[(.*?)\]\((?!https)(.*?)\)"
        for k,v in re.findall(pattern, text):
            if 'http' not in v and len(k.split())<=3 and v.split('/')[-1].startswith('#'):
                hyperlinks[k] = v
        return hyperlinks
    
    def load_data(
        self, file: Path, extra_info: Optional[Dict] = None
    ) -> List[Document]:
        """Parse file into string."""
        tups = self.parse_tups(file)
        results = []
        
        text = '\n'.join([f"\n\n{header}\n{value}" for header,value in tups])
        local_metadata = self.get_local_metadata(text)

        text = self.remove_hyperlinks(text)
        extra_info = dict(extra_info,hyperlinks=local_metadata) if extra_info else {}
        return [Document(text=text,metadata=extra_info)]
        
        


In [7]:
md_reader = RagasMdReader(remove_hyperlinks=False)

In [8]:
def llm2(prompt, **kwargs):
    response = openai.ChatCompletion.create(
        model=kwargs.get("model", "gpt-4"),
        messages=[{"role": "system", "content": prompt}],
        temperature=kwargs.get("temperature", 0),
        top_p=kwargs.get("top_p", 1),
        frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        presence_penalty=kwargs.get("presence_penalty", 0.0),
        max_tokens=kwargs.get("max_tokens", 500),
        n=kwargs.get("n", 1),
    )
    return response

In [9]:
from ragas.testset.prompts import SEED_QUESTION, EVOLUTION_ELIMINATION

In [10]:
dirs = [
    "/Users/shahules/Myprojects/rag-experiments/gitlab-handbook/data/handbook/leadership/",
    "/Users/shahules/Myprojects/rag-experiments/gitlab-handbook/data/handbook/company/",
]
documents = []
for dir_path in dirs:
    loader = SimpleDirectoryReader(dir_path, 
                                   recursive=True,
                                  file_extractor={".md":md_reader},
                                file_metadata=RagasMdReader.get_file_metadata)
    
    documents.extend(loader.load_data())

In [11]:
len(documents)

168

In [13]:
# documents[0].metadata

In [12]:
testset = testsetgenerator.generate(documents, test_size=50)


  0%|                                                            | 0/50 [00:00<?, ?it/s]

{'score': True, 'is_table_present': False}
seed question What are some strategies to build influence in the workplace?


  2%|█                                                   | 1/50 [00:42<34:59, 42.85s/it]/Users/shahules/belar/src/ragas/testset/testset_generator.py:302: UserWarning: No neighbors exists
  warnings.warn("No neighbors exists")


{'score': True, 'is_table_present': True}
seed question Who is the facilitator of the expense management working group?
{'score': True, 'is_table_present': False}
seed question What is the first phase of the three year strategy process?
{'score': True, 'is_table_present': False}
seed question What is GitLab's strategic goal?


  6%|███                                                 | 3/50 [01:17<18:45, 23.94s/it]

...

1176it [37:24,  1.38it/s]

{'score': True, 'is_table_present': True}
seed question What were the main areas of improvement identified by the working group in the Object Storage implementation?
{'score': True, 'is_table_present': True}
seed question Who is the Triage Lead in the Working Group?
{'score': True, 'is_table_present': True}
seed question What is the due date for the "Definition of the objects" phase in the roadmap?
{'score': True, 'is_table_present': False}
seed question What is the purpose of adding new models to the CustomersDot data architecture?
{'score': True, 'is_table_present': True}
seed question What is the target end date for the GitHost migration?
{'score': True, 'is_table_present': True}
seed question Who is the Senior Director of Development in the Working Group?
{'score': True, 'is_table_present': True}
seed question What tasks are still outstanding at the time of closing this Working Group?
{'score': True, 'is_table_present': True}
seed question What is the role of Farnoosh Seifoddini in

1225it [40:29,  1.69s/it]

{'score': True, 'is_table_present': False}
seed question What is the role of a leader at GitLab in building high-performing teams?


1275it [40:55,  1.93s/it]


In [13]:
testset.to_pandas().to_csv("gitlab_communication_company_v2.csv")

In [23]:
testset.to_pandas()

,seed_question,question,context,answer,question_type,episode_done,evolution_elimination
0,What are some strategies to build influence in...,How can communication patterns be utilized to ...,"- ""During a crucial conversation, different pr...",Communication patterns can be utilized to buil...,conditional,False,False
1,What are the strategic goals of GitLab?,What is GitLab's strategic goal to become the ...,- Along the road to realizing our mission of e...,GitLab's strategic goal to become the leading ...,reasoning,True,False
2,What functionality aligns with the reporter pe...,Which GitLab permission allows for managing la...,The main functionality that some non-developer...,The GitLab permission that allows for managing...,reasoning,True,False
3,What are the different scopes for caching avai...,What are the different caching scopes availabl...,"1. ""In order to reduce the database overhead, ...",The different caching scopes available for red...,conditional,False,False
4,What is the purpose of the Quote-to-Cash process?,What is the purpose of the Quote-to-Cash proce...,- An efficient Quote-to-Cash system makes purc...,The purpose of the Quote-to-Cash process is to...,multi_context,True,False
5,Who is responsible for running meetings in a w...,Who facilitates meetings and ensures operation...,- A facilitator is responsible for running mee...,The facilitator is responsible for running mee...,multi_context,True,False
6,What are some suggested team building activiti...,What team building activities can be used to b...,"- ""It's great for building trust, getting to k...",Some team building activities that can be used...,multi_context,True,True
7,What is the impact of elevating the caching le...,What is the impact of implementing a per-proce...,- When we started to cache license information...,The impact of implementing a per-process memor...,reasoning,True,True
8,What should I review before compensation conve...,What should I review before compensation conve...,- Conversations with regards to compensation a...,"Before compensation conversations at GitLab, y...",simple,True,None
9,What is the objective of establishing a repeat...,What is the objective of establishing a repeat...,- We aim to create a working group to establis...,The objective of establishing a repeatable pro...,simple,True,None


In [20]:
q1="How should I use my notification settings in Slack?"
q2="What's the best way to manage Slack notification settings for efficient communication and minimal disruptions?"


In [21]:
prompt = EVOLUTION_ELIMINATION.prompt.template.format(question1=q1, question2=q2)

In [22]:
llm2(prompt)

<OpenAIObject chat.completion id=chatcmpl-8JFEQOhc3FPRQ6hhpviDo4mVdSD9G at 0x7f944a3f65c0> JSON: {
  "id": "chatcmpl-8JFEQOhc3FPRQ6hhpviDo4mVdSD9G",
  "object": "chat.completion",
  "created": 1699597474,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"reason\": \"Both questions are asking about managing notification settings in Slack, but the second question adds additional requirements of efficient communication and minimal disruptions, which changes the depth of the inquiry.\",\n  \"verdict\": \"Not Equal\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 212,
    "completion_tokens": 49,
    "total_tokens": 261
  }
}